In [16]:
import os
import sys
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from tqdm.notebook import tqdm
from collections import namedtuple
from collections import defaultdict
from pprint import pprint
import random
import pickle
from functools import reduce

# Configure display options
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
%config InlineBackend.figure_format = 'retina'

# Add project directory to sys.path
# pdir = os.path.abspath(os.path.join(os.getcwd(), '.'))
# sys.path.insert(1, pdir)
# from myutils import *

# Other module imports
import ast, math, swifter, csv, json, itertools as it, portion as P

# Set plot style
# plt.style.use('ggplot')

In [17]:
def set_data(df, mode='pcap', tz=0):
    def nr_serv_cel(row):
        pos = row.serv_cel_pos
        if pos == 255:
            return 65535, -160, -50
        else:
            return row[f'PCI{pos}'], row[f'RSRP{pos}'], row[f'RSRQ{pos}']
    
    if mode == 'pcap':
        common_column_names = ['seq', 'rpkg', 'frame_id', 'Timestamp', 'lost', 'excl', 'latency', 'xmit_time', 'arr_time']
        
        if df.empty:
            return pd.DataFrame(columns=common_column_names)
        
        date_columns = ['Timestamp', 'xmit_time', 'arr_time']
        df[date_columns] = df[date_columns].apply(pd.to_datetime)
        df[['seq', 'rpkg', 'frame_id']] = df[['seq', 'rpkg', 'frame_id']].astype('Int32')
        df[['latency']] = df[['latency']].astype('float32')
        df[['lost', 'excl']] = df[['lost', 'excl']].astype('boolean')

    if mode in ['lte', 'nr']:
        common_column_names = [
            'Timestamp', 'type_id', 'PCI', 'RSRP', 'RSRQ', 'serv_cel_index', 'EARFCN', 'NR_ARFCN', 
            'num_cels', 'num_neigh_cels', 'serv_cel_pos', 'PCI0', 'RSRP0', 'RSRQ0',
        ]
        
        if df.empty:
            return pd.DataFrame(columns=common_column_names)
        
        if mode == 'lte':
            columns_mapping = {
                'RSRP(dBm)': 'RSRP',
                'RSRQ(dB)': 'RSRQ',
                'Serving Cell Index': 'serv_cel_index',
                'Number of Neighbor Cells': 'num_neigh_cels',
                'Number of Detected Cells': 'num_cels',
            }
            columns_order = [*common_column_names, *df.columns[df.columns.get_loc('PCI1'):].tolist()]
            
            df = df.rename(columns=columns_mapping).reindex(columns_order, axis=1)
            df['serv_cel_index'] = np.where(df['serv_cel_index'] == '(MI)Unknown', '3_SCell', df['serv_cel_index'])
            df['num_cels'] = df['num_neigh_cels'] + 1
            df['type_id'] = 'LTE_PHY'

        if mode == 'nr':
            columns_mapping = {
                'Raster ARFCN': 'NR_ARFCN',
                'Serving Cell Index': 'serv_cel_pos',
                'Num Cells': 'num_cels',
            }
            columns_order = [*common_column_names, *df.columns[df.columns.get_loc('PCI1'):].tolist()]
            
            df = df.rename(columns=columns_mapping).reindex(columns_order, axis=1)
            df[['PCI', 'RSRP', 'RSRQ']] = df.apply(nr_serv_cel, axis=1, result_type='expand')
            df['serv_cel_index'] = np.where(df['serv_cel_pos'] == 255, df['serv_cel_index'], 'PSCell')
            df['num_neigh_cels'] = np.where(df['serv_cel_pos'] == 255, df['num_cels'], df['num_cels'] - 1)
            df['type_id'] = '5G_NR_ML1'
        
        df['Timestamp'] = pd.to_datetime(df['Timestamp']) + pd.Timedelta(hours=tz)
        df[['type_id', 'serv_cel_index']] = df[['type_id', 'serv_cel_index']].astype('category')
        df[['EARFCN', 'NR_ARFCN']] = df[['EARFCN', 'NR_ARFCN']].astype('Int32')
        df[['num_cels', 'num_neigh_cels', 'serv_cel_pos']] = df[['num_cels', 'num_neigh_cels', 'serv_cel_pos']].astype('UInt8')

        for tag in df.columns:
            if tag.startswith('PCI'):
                df[tag] = df[tag].astype('Int32')
            if tag.startswith(('RSRP', 'RSRQ')):
                df[tag] = df[tag].astype('float32')

    return df

In [18]:
import os
import json

# Specify root directory
root_dir = "/Users/jackbedford/Desktop/MOXA/Code/data"

# Collect dates
dates = [s for s in sorted(os.listdir(root_dir)) if os.path.isdir(os.path.join(root_dir, s)) and s != 'backup']
date_dirs = [os.path.join(root_dir, s) for s in dates]

for date, date_dir in zip(dates, date_dirs):
    # Specify the path to your JSON file
    date = os.path.basename(date_dir)
    json_filepath = os.path.join(date_dir, f'{date}.json')
    
    # Read the JSON file and load its contents into a dictionary
    with open(json_filepath, 'r', encoding='utf-8') as json_file:
        my_dict = json.load(json_file)
    
    if not my_dict:
        continue
        
    print(date, len(my_dict))
    
    for exp, item in my_dict.items():
        print({exp: item})

2023-03-16 1
{'UDP_Bandlock_B1_B7_B8_RM500Q': {'route': 'BR', 'devices': {'qc00': 'B1', 'qc02': 'B7', 'qc03': 'B8'}, 'ods': [[0, 'Origin Station', 'Destination Station', 'Departure Time', 'Notes'], [1, '萬芳醫院', '動物園', '19:23', ''], [2, '動物園', '萬芳醫院', '19:32', ''], [3, '萬芳醫院', '動物園', '19:39', ''], [4, '動物園', '萬芳醫院', '19:47', ''], [5, '萬芳醫院', '動物園', '19:55', ''], [6, '動物園', '萬芳醫院', '20:03', ''], [7, '萬芳醫院', '動物園', '20:09', ''], [8, '動物園', '萬芳醫院', '20:18', ''], [9, '萬芳醫院', '動物園', '20:27', ''], [10, '動物園', '萬芳醫院', '20:33', ''], [11, '萬芳醫院', '動物園', '20:42', ''], [12, '動物園', '萬芳醫院', '20:49', '']], 'stations': ['動物園', '木柵', '萬芳社區', '萬芳醫院'], 'telecom': 'CHT', 'protocol': 'UDP', 'length': [250, 'byte'], 'bitrate': [1, 'Mbps'], 'notes': ''}}
2023-03-26 1
{'UDP_Bandlock_All_RM500Q': {'route': 'BR', 'devices': {'qc00': 'All', 'qc02': 'All', 'qc03': 'All'}, 'ods': [[0, 'Origin Station', 'Destination Station', 'Departure Time', 'Notes'], [1, '萬芳醫院', '動物園', '16:36', ''], [2, '動物園', '萬芳醫院', '16:46', ''

In [19]:
# Select Dates
# sel_dates = dates[:]
# sel_dates = [s for s in dates if s >= '2023-09-12']
# sel_dates = [s for s in dates if s < '2023-09-12']
# sel_dates = [s for s in dates if s >= '2023-08-16' and s < '2023-09-12']
# sel_dates = dates[:5]
# sel_dates = dates[-8:]
sel_dates = ['2023-09-12-2']
exc_dates = []
# exc_dates = ['2023-06-12', '2023-06-13']
date_dirs = [os.path.join(root_dir, s) for s in sel_dates if s not in exc_dates]

# Select Experiment Names
sel_exps = []
# sel_exps = ['UDP_Bandlock_9S_Phone_Brown', 'UDP_Bandlock_9S_Phone_Airport']
exc_exps = []
# exc_exps = ['Modem_Action_Test', 'Control_Group', 'Control_Group2', 'Control_Group3']

# Select Routes
sel_routes = []
# sel_routes = ['BR']

In [20]:
# Collect Experiments
exps_dict = {}

for date_dir in date_dirs:
    date = os.path.basename(date_dir)
    
    # Specify the path to your JSON file
    json_filepath = os.path.join(date_dir, f'{date}.json')
    
    # Read the JSON file and load its contents into a dictionary
    with open(json_filepath, 'r', encoding='utf-8') as json_file:
        my_dict = json.load(json_file)
    
    if not my_dict:
        continue
    
    for i, (exp, item) in enumerate(my_dict.items()):
        if len(sel_exps) != 0 and exp not in sel_exps:
            continue
        
        if len(exc_exps) != 0 and exp in exc_exps:
            continue
        
        if len(sel_routes) != 0 and item['route'] not in sel_routes:
            continue
        
        try:
            exps_dict[date] = {**exps_dict[date], **{exp: item}}
        except:
            exps_dict[date] = {exp: item}

for date, exps in exps_dict.items():
    print(date, len(exps))
    
    for exp_name, exp in exps.items():
        print({exp_name: exp})

2023-09-12-2 1
{'UDP_Bandlock_9S_Phone_Brown': {'skip': False, 'route': 'BR', 'devices': {'sm00': 'All', 'sm01': 'All', 'sm02': 'B3', 'sm03': 'B7', 'sm04': 'B8', 'sm05': 'B3B7', 'sm06': 'B3B8', 'sm07': 'B7B8', 'sm08': 'LTE'}, 'ods': [[0, 'Origin Station', 'Destination Station', 'Departure Time', 'Notes'], [1, '動物園', '南港展覽館', '13:34', '到木柵才想起要開GPS'], [2, '南港展覽館', '動物園', '14:52', '']], 'stations': ['動物園', '木柵', '萬芳社區', '萬芳醫院', '辛亥', '麟光', '六張犁', '科技大樓', '大安', '忠孝復興', '南京復興', '中山國中', '松山機場', '大直', '劍南路', '西湖', '港墘', '文德', '內湖', '大湖公園', '葫洲', '東湖', '南港軟體園區', '南港展覽館'], 'ue': 'Phone', 'sync': None, 'telecom': 'CHT', 'protocol': 'UDP', 'length': [250, 'byte'], 'bitrate': [1, 'Mbps'], 'notes': ''}}


In [21]:
for date, exps in exps_dict.items():
    print(date, len(exps))
    
    for exp_name, exp in exps.items():
        exp_dir = os.path.join(root_dir, date, exp_name)
        print({exp_name: exp})
        
        devices = list(exp['devices'].keys())
        trips = ['#{:02d}'.format(s[0]) for s in exp['ods'][1:]]
        for trip in trips:
            for dev in devices:
                data_dir = os.path.join(exp_dir, dev, trip, 'data')
                output_filepath = os.path.join(data_dir, 'experiment_duration.csv')
                
                print(output_filepath, os.path.isfile(output_filepath))
                # os.system(f'rm {output_filepath}')
                if os.path.isfile(output_filepath):
                    continue
                
                try:
                    dl_filepath = [os.path.join(data_dir, s) for s in os.listdir(data_dir) if s.startswith('udp_dnlk_loss_latency_ho.pkl')][0]
                    # print(dl_filepath, os.path.isfile(dl_filepath))
                    dl_df = pd.read_pickle(dl_filepath)
                except:
                    dl_filepath = [os.path.join(data_dir, s) for s in os.listdir(data_dir) if s.startswith('udp_dnlk_loss_latency.csv')][0]
                    # print(dl_filepath, os.path.isfile(dl_filepath))
                    dl_df = set_data(pd.read_csv(dl_filepath))
                    
                try:
                    ul_filepath = [os.path.join(data_dir, s) for s in os.listdir(data_dir) if s.startswith('udp_uplk_loss_latency_ho.pkl')][0]
                    # print(ul_filepath, os.path.isfile(ul_filepath))
                    ul_df = pd.read_pickle(ul_filepath)
                except:
                    ul_filepath = [os.path.join(data_dir, s) for s in os.listdir(data_dir) if s.startswith('udp_uplk_loss_latency.csv')][0]
                    # print(ul_filepath, os.path.isfile(ul_filepath))
                    ul_df = set_data(pd.read_csv(ul_filepath))
                
                start_ts = min([dl_df['Timestamp'].iloc[0], ul_df['Timestamp'].iloc[0]])
                end_ts = max([dl_df['Timestamp'].iloc[-1], ul_df['Timestamp'].iloc[-1]])
                exp_duration = round((end_ts - start_ts).total_seconds() / 60, 2)  # (minutes)
                
                rrc_filepath = [os.path.join(data_dir, s) for s in os.listdir(data_dir) if s.startswith('diag_log') and s.endswith('rrc.csv')][0]
                # print(rrc_filepath, os.path.isfile(rrc_filepath))
                rrc_df = pd.read_csv(rrc_filepath, parse_dates=['Timestamp'], low_memory=False)
                rrc_df["Timestamp"] = rrc_df["Timestamp"].swifter.apply(lambda x: x + dt.timedelta(hours=8))
                rrc_df = rrc_df[(rrc_df['Timestamp'] >= start_ts) & (rrc_df['Timestamp'] < end_ts)].reset_index(drop=True)
                if not rrc_df.empty:
                    rrc_start_ts = rrc_df['Timestamp'].iloc[0] - pd.Timedelta(seconds=1)
                    rrc_end_ts = rrc_df['Timestamp'].iloc[-1] + pd.Timedelta(seconds=1)
                    rrc_duration = round((rrc_end_ts - rrc_start_ts).total_seconds() / 60, 2)  # (minutes)
                else:
                    rrc_start_ts = pd.NA
                    rrc_end_ts = pd.NA
                    rrc_duration = 0.00 
                
                fieldnames=['No.', 'Start', 'End', 'Duration']
                data = [
                    {'No.': 'total', 'Start': start_ts, 'End': end_ts, 'Duration': exp_duration},
                    {'No.': 'valid', 'Start': rrc_start_ts, 'End': rrc_end_ts, 'Duration': rrc_duration},
                ]
                
                with open(output_filepath, 'w', newline='') as csvfile:
                    csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                    csv_writer.writeheader()
                    csv_writer.writerows(data)

2023-09-12-2 1
{'UDP_Bandlock_9S_Phone_Brown': {'skip': False, 'route': 'BR', 'devices': {'sm00': 'All', 'sm01': 'All', 'sm02': 'B3', 'sm03': 'B7', 'sm04': 'B8', 'sm05': 'B3B7', 'sm06': 'B3B8', 'sm07': 'B7B8', 'sm08': 'LTE'}, 'ods': [[0, 'Origin Station', 'Destination Station', 'Departure Time', 'Notes'], [1, '動物園', '南港展覽館', '13:34', '到木柵才想起要開GPS'], [2, '南港展覽館', '動物園', '14:52', '']], 'stations': ['動物園', '木柵', '萬芳社區', '萬芳醫院', '辛亥', '麟光', '六張犁', '科技大樓', '大安', '忠孝復興', '南京復興', '中山國中', '松山機場', '大直', '劍南路', '西湖', '港墘', '文德', '內湖', '大湖公園', '葫洲', '東湖', '南港軟體園區', '南港展覽館'], 'ue': 'Phone', 'sync': None, 'telecom': 'CHT', 'protocol': 'UDP', 'length': [250, 'byte'], 'bitrate': [1, 'Mbps'], 'notes': ''}}
/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12-2/UDP_Bandlock_9S_Phone_Brown/sm00/#01/data/experiment_duration.csv False
/Users/jackbedford/Desktop/MOXA/Code/data/2023-09-12-2/UDP_Bandlock_9S_Phone_Brown/sm01/#01/data/experiment_duration.csv False
/Users/jackbedford/Desktop/MOXA/Code/data/2023

In [22]:
for date, exps in exps_dict.items():
    print(date, len(exps))
    
    for exp_name, exp in exps.items():
        exp_dir = os.path.join(root_dir, date, exp_name)
        print({exp_name: exp})
        
        # Check data abnormality
        exp_duration_check_path = os.path.join(exp_dir, 'exp_duration_minutes_check.csv')
        rrc_duration_check_path = os.path.join(exp_dir, 'rrc_duration_minutes_check.csv')
        nr_duration_check_path = os.path.join(exp_dir, 'nr_duration_minutes_check.csv')
        nr_alive_check_path = os.path.join(exp_dir, 'nr_alive_check.csv')
        
        # print(exp_duration_check_path, os.path.isfile(exp_duration_check_path))
        # print(rrc_duration_check_path, os.path.isfile(rrc_duration_check_path))
        # print(nr_alive_check_path, os.path.isfile(nr_alive_check_path))
        # print(nr_duration_check_path, os.path.isfile(nr_duration_check_path))
        
        # os.system(f'rm {exp_duration_check_path}')
        # os.system(f'rm {rrc_duration_check_path}')
        # os.system(f'rm {nr_alive_check_path}')
        # os.system(f'rm {nr_duration_check_path}')
        
        if os.path.isfile(exp_duration_check_path) and os.path.isfile(rrc_duration_check_path) and os.path.isfile(nr_duration_check_path) and os.path.isfile(nr_alive_check_path):
            exp_duration_table = pd.read_csv(exp_duration_check_path)
            rrc_duration_table = pd.read_csv(rrc_duration_check_path)
            nr_duration_table = pd.read_csv(nr_duration_check_path)
            nr_alive_table = pd.read_csv(nr_alive_check_path)
        else:
            devices = list(exp['devices'].keys())
            trips = ['#{:02d}'.format(s[0]) for s in exp['ods'][1:]]
            
            exp_duration_table = pd.DataFrame(columns=['Tr.', *devices])
            rrc_duration_table = pd.DataFrame(columns=['Tr.', *devices])
            nr_duration_table = pd.DataFrame(columns=['Tr.', *devices])
            nr_alive_table = pd.DataFrame(columns=['Tr.', *devices])

            for trip in trips:
                exp_duration_list = [trip] + [0.00] * len(devices)
                rrc_duration_list = [trip] + [0.00] * len(devices)
                nr_duration_list = [trip] + [0.00] * len(devices)
                nr_alive_list = [trip] + ['O'] * len(devices)
                
                for j, dev in enumerate(devices):
                    data_dir = os.path.join(exp_dir, dev, trip, 'data')
                    dur_filepath = [os.path.join(data_dir, s) for s in os.listdir(data_dir) if s.startswith('experiment_duration.csv')][0]
                    rrc_filepath = [os.path.join(data_dir, s) for s in os.listdir(data_dir) if s.startswith('diag_log') and s.endswith('rrc.csv')][0]
                    nr_filepath = [os.path.join(data_dir, s) for s in os.listdir(data_dir) if s.startswith('diag_log') and s.endswith('nr_ml1.csv')][0]
                    
                    # print(dur_filepath, os.path.isfile(dur_filepath))
                    # print(rrc_filepath, os.path.isfile(rrc_filepath))
                    # print(nr_filepath, os.path.isfile(nr_filepath))
                    
                    dur_df = pd.read_csv(dur_filepath, parse_dates=['Start', 'End'])
                    start_ts = dur_df['Start'].iloc[0] - pd.Timedelta(hours=8)
                    end_ts = dur_df['End'].iloc[0] - pd.Timedelta(hours=8)
                    exp_duration_list[j+1] = dur_df['Duration'].iloc[0]
                    
                    rrc_df = pd.read_csv(rrc_filepath, parse_dates=['Timestamp'], low_memory=False)
                    rrc_df = rrc_df[(rrc_df['Timestamp'] >= start_ts) & (rrc_df['Timestamp'] < end_ts)].reset_index(drop=True)
                    if not rrc_df.empty:
                        rrc_start_ts = rrc_df['Timestamp'].iloc[0]
                        rrc_end_ts = rrc_df['Timestamp'].iloc[-1]
                        rrc_duration_list[j+1] = round((rrc_end_ts - rrc_start_ts).total_seconds() / 60, 2)  # (minutes)
                    
                    if rrc_df.empty:
                        nr_alive_list[j+1] = '-'
                    elif sum(np.in1d(rrc_df['failureType-r15'], 'rlc-MaxNumRetx (2)')) > 0:
                        nr_alive_list[j+1] = '∆'
                    
                    nr_df = pd.read_csv(nr_filepath, parse_dates=['Timestamp'])
                    nr_df = nr_df[(nr_df['Timestamp'] >= start_ts) & (nr_df['Timestamp'] < end_ts)].reset_index(drop=True)
                    if rrc_df.empty and nr_df.empty:
                        nr_alive_list[j+1] = '-'
                    elif nr_df.empty:
                        nr_alive_list[j+1] = 'X'
                    
                    if not nr_df.empty:
                        nr_start_ts = nr_df['Timestamp'].iloc[0]
                        nr_end_ts = nr_df['Timestamp'].iloc[-1]
                        nr_duration_list[j+1] = round((nr_end_ts - nr_start_ts).total_seconds() / 60, 2)  # (minutes)
                
                exp_duration_table.loc[len(exp_duration_table)] = exp_duration_list
                rrc_duration_table.loc[len(rrc_duration_table)] = rrc_duration_list
                nr_duration_table.loc[len(nr_duration_table)] = nr_duration_list
                nr_alive_table.loc[len(nr_alive_table)] = nr_alive_list
            
            exp_duration_table.to_csv(exp_duration_check_path, index=False)
            rrc_duration_table.to_csv(rrc_duration_check_path, index=False)
            nr_duration_table.to_csv(nr_duration_check_path, index=False)
            nr_alive_table.to_csv(nr_alive_check_path, index=False)
        
        print('exp_duration_table (minutes)')
        display(exp_duration_table)
        print('rrc_duration_table (minutes)')
        display(rrc_duration_table)
        print('nr_duration_table (minutes)')
        display(nr_duration_table)
        print('nr_alive_table')
        display(nr_alive_table)

print('O: RF Module is always on.')
print('∆: RF Module was turned off midway. "SCG Failure: rlc-MaxNumRetx (2)" occurs!')
print('X: RF Module has never been turned on. It could be under pure LTE or an anomaly.')
print('-: RRC File is EMPTY!!!')

2023-09-12-2 1
{'UDP_Bandlock_9S_Phone_Brown': {'skip': False, 'route': 'BR', 'devices': {'sm00': 'All', 'sm01': 'All', 'sm02': 'B3', 'sm03': 'B7', 'sm04': 'B8', 'sm05': 'B3B7', 'sm06': 'B3B8', 'sm07': 'B7B8', 'sm08': 'LTE'}, 'ods': [[0, 'Origin Station', 'Destination Station', 'Departure Time', 'Notes'], [1, '動物園', '南港展覽館', '13:34', '到木柵才想起要開GPS'], [2, '南港展覽館', '動物園', '14:52', '']], 'stations': ['動物園', '木柵', '萬芳社區', '萬芳醫院', '辛亥', '麟光', '六張犁', '科技大樓', '大安', '忠孝復興', '南京復興', '中山國中', '松山機場', '大直', '劍南路', '西湖', '港墘', '文德', '內湖', '大湖公園', '葫洲', '東湖', '南港軟體園區', '南港展覽館'], 'ue': 'Phone', 'sync': None, 'telecom': 'CHT', 'protocol': 'UDP', 'length': [250, 'byte'], 'bitrate': [1, 'Mbps'], 'notes': ''}}
exp_duration_table (minutes)


,Tr.,sm00,sm01,sm02,sm03,sm04,sm05,sm06,sm07,sm08
0,#01,47.26,47.26,47.26,47.26,47.26,47.26,47.26,47.26,47.26
1,#02,48.31,48.30,48.30,48.30,48.30,48.31,48.30,48.30,48.30


rrc_duration_table (minutes)


,Tr.,sm00,sm01,sm02,sm03,sm04,sm05,sm06,sm07,sm08
0,#01,47.04,47.02,47.19,47.21,47.26,47.19,47.16,47.09,47.20
1,#02,47.90,47.91,47.87,47.92,47.89,47.77,47.91,47.92,48.23


nr_duration_table (minutes)


,Tr.,sm00,sm01,sm02,sm03,sm04,sm05,sm06,sm07,sm08
0,#01,44.22,40.60,36.47,24.55,21.68,24.25,23.70,17.13,0.0
1,#02,47.90,13.94,15.94,18.86,47.89,39.82,46.15,17.67,0.0


nr_alive_table


,Tr.,sm00,sm01,sm02,sm03,sm04,sm05,sm06,sm07,sm08
0,#01,∆,∆,∆,∆,∆,∆,∆,∆,X
1,#02,∆,∆,∆,∆,∆,∆,∆,∆,X


O: RF Module is always on.
∆: RF Module was turned off midway. "SCG Failure: rlc-MaxNumRetx (2)" occurs!
X: RF Module has never been turned on. It could be under pure LTE or an anomaly.
-: RRC File is EMPTY!!!
